# All The reports and comments are here alongside the code just to keep everything in one place

### Search Algorithms used in 8-puzzle
- **State**: is the order of the number in the 3*3 matrix

- **Actions**: is swapping the `0` with any of its neighbours

- **Goal state**: to reach the below matrix.

- **Path cost**: Total number of moves(swaps)

- **Transition model**: swap the `0` with the corresponding neighbour 

First we import some usefull libs

[1, 2, 3]<br/>
[4, 5, 6]<br/>
[7, 8, 0]<br/>

In [42]:
import time
import random
from typing import Tuple, Iterable
from copy import deepcopy

Then add some constants

In [43]:
EASY_INSTANCES =   [[ [1,2,3],[0,7,6],[5,4,8] ],
                    [ [0,4,1],[2,5,3],[7,8,6] ],
                    [ [4,1,3],[0,2,6],[7,5,8] ],
                    [ [1,2,3],[0,4,8],[7,6,5] ],
                    [ [1,2,0],[4,8,3],[7,6,5] ],
                    [ [1,0,2],[4,6,3],[7,5,8] ],
                    [ [0,1,2],[4,5,3],[7,8,6] ],
                    [ [1,2,3],[0,4,5],[7,8,6] ],
                    [ [1,2,3],[4,0,5],[7,8,6] ],
                    [ [1,2,3],[4,5,0],[7,8,6] ],
                    [ [0,1,3],[4,2,5],[7,8,6] ],
                    [ [2,3,5],[1,0,4],[7,8,6] ],
                    [ [1,6,2],[5,3,0],[4,7,8] ],
                    [ [1,8,2],[0,4,3],[7,6,5] ],
                    [ [2,5,3],[4,1,6],[0,7,8] ],
                    [ [1,2,3],[4,6,8],[7,5,0] ],
                    [ [1,6,2],[5,7,3],[0,4,8] ],
                    [ [0,4,1],[5,3,2],[7,8,6] ],
                    [ [0,5,2],[1,8,3],[4,7,6] ],
                    [ [1,2,3],[0,4,6],[7,5,8] ]]

MEDIUM_INSTANCES = [[ [1,3,5],[7,2,6],[8,0,4] ],
                    [ [4,1,2],[3,0,6],[5,7,8] ],
                    [ [4,3,1],[0,7,2],[8,5,6] ],
                    [ [5,2,1],[4,8,3],[7,6,0] ],
                    [ [2,0,8],[1,3,5],[4,6,7] ],
                    [ [3,5,6],[1,4,8],[0,7,2] ],
                    [ [1,0,2],[7,5,4],[8,6,3] ],
                    [ [5,1,8],[2,7,3],[4,0,6] ],
                    [ [4,3,0],[6,1,8],[2,7,5] ],
                    [ [2,4,3],[1,6,5],[8,0,7] ],
                    [ [1,2,3],[6,4,5],[7,8,0] ],
                    [ [3,1,2],[4,5,6],[7,8,0] ],
                    [ [1,2,3],[4,8,7],[6,5,0] ],
                    [ [1,3,2],[5,4,6],[7,8,0] ],
                    [ [1,4,2],[6,5,8],[7,3,0] ]]

HARD_INSTANCES =   [[ [2,1,3],[4,5,6],[8,7,0] ],
                    [ [2,3,1],[6,5,4],[8,7,0] ],
                    [ [2,3,1],[6,4,5],[7,8,0] ],
                    [ [1,2,3],[6,5,4],[8,7,0] ],
                    [ [1,2,3],[6,5,4],[0,8,7] ],
                    [ [4,5,3],[2,8,0],[6,7,1] ],
                    [ [4,5,3],[2,1,0],[8,7,6] ],
                    [ [1,2,4],[3,5,0],[8,7,6] ],
                    [ [1,2,4],[3,5,8],[7,0,6] ],
                    [ [2,1,3],[4,5,8],[7,0,6] ],
                    [ [1,3,5],[8,7,0],[6,2,4] ],
                    [ [4,3,1],[6,5,8],[0,2,7] ],
                    [ [7,0,4],[8,5,1],[6,3,2] ],
                    [ [8,7,2],[1,5,0],[4,6,3] ],
                    [ [8,3,5],[6,4,2],[1,0,7] ],
                    [ [1,6,4],[0,3,5],[8,2,7] ],
                    [ [6,3,8],[5,4,1],[7,2,0] ],
                    [ [5,8,7],[1,4,6],[3,0,2] ],
                    [ [2,8,5],[3,6,1],[7,0,4] ],
                    [ [8,7,6],[5,4,3],[2,1,0] ]]
                        
RUN_ALL = True
                        
RIGHT = 'R'
LEFT = 'L'
UP = 'U'
DOWN = 'D'

### heuristic function
One of the best heuristic function is the function that calculates all distances for each number from the place it should be.

Another option would be the total number of numbers that are not in the place they should be. But as we've proved in the course the first one is way better and more realistic.

In [44]:
N = 3
def h(board: list[list[int]]):
    hValue = 0
    for i, row in enumerate(board):
        for j, num in enumerate(row):
            if num != 0:
                wantedRow = (num-1)//N
                wantedColumn = (num-1)%N
                hValue += abs(i - wantedRow)
                hValue += abs(j - wantedColumn)
            if num == 0:
                hValue += abs(i - (N - 1))
                hValue += abs(j - (N - 1))
    return hValue

#### Here's some usefun functions. They are also self-descriptive :)

In [45]:
def findElement(board: Iterable[Iterable[int]], targetNum: int):
    for i, row in enumerate(board):
        for j, num in enumerate(row):
            if num == targetNum:
                return i, j
            
def isAnswer(board: Iterable[Iterable[int]]) -> bool:
    checker = 1
    for row in board:
        for num in row:
            if(checker == 9 and num == 0):
                return True
            if num != checker:
                return False
            checker += 1
    return False      

def toTuple(board: Iterable[Iterable[int]]):
    return (tuple(board[0]),
            tuple(board[1]),
            tuple(board[2]))

def swapItems(board: list[list[int]], coord1: Tuple[int], coord2: Tuple[int]):
    temp = board[coord1[0]][coord1[1]]
    
    board[coord1[0]][coord1[1]] = board[coord2[0]][coord2[1]]
    
    board[coord2[0]][coord2[1]] = temp
              

### To have a simple tree data structure we created a `TreeNode` class that hold anything necessary like `parent`, list of `children`, it's `board` represent the state of the node and other stuff like `action` that was taken to reach this node and it's depth.
#### **Note that to use these `TreeNode`s in a `PriorityQueue` we have to overload some comparetive operators**

In [46]:
class TreeNode:
    def __init__(self, parent, board, action: str) -> None:
        parent: TreeNode
        self.parent = parent
        self.children = []
        self.board = board
        self.depth = parent.depth + 1 if parent else 0
        self.action = action
        self.h = -1 # heuristic value
        
        
    def __repr__(self) -> str:
        return self.__str__()
    
    def __str__(self) -> str:
        return f"Node of {self.board}"
    
    def f(self):
        if self.h == -1:
            self.h = h(self.board)
                
        return self.depth + self.h # g(node) + h(node)
    
    """
    The operator overloadings
    """
    
    def __lt__(self, other):
        if isinstance(other, TreeNode):
            return self.f() < other.f()
        
        raise TypeError(f"Unknown comparison between TreeNode and {type(other).__name__}")
    
    def __le__(self, other):
        if (isinstance(other, TreeNode)):
            return self.f() <= other.f()
        
        raise TypeError(f"Unknown comparison between TreeNode and {type(other).__name__}")
        
    
    def __gt__(self, other):
        if isinstance(other, TreeNode):
            return self.f() > other.f()
        
        raise TypeError(f"Unknown comparison between TreeNode and {type(other).__name__}")
        
    def __ge__(self, other):
        if (isinstance(other, TreeNode)):
            return self.f() >= other.f()
        
        raise TypeError(f"Unknown comparison between TreeNode and {type(other).__name__}")
        
        
    def __eq__(self, other):
        if (isinstance(other, TreeNode)):
            return self.f() == other.f()
        
        raise TypeError(f"Unknown comparison between TreeNode and {type(other).__name__}")
                

### And here are some useful function to make things easier while running a few samples having the algorithm(`solver`) as one of the factors

In [47]:
def printBoardNicely(board: list):
    print(*board, sep='\n')
    
def traverseToRoot(leafNode: TreeNode):
    trackingList = []
    itter = leafNode
    while itter:
        trackingList.append((itter.action, itter.board))
        itter = itter.parent
        
    trackingList.reverse()
    return trackingList

def printTrackingNicely(trackingList: list):
    print(f"From root:")
    printBoardNicely(trackingList[0][1])
    for record in trackingList[1:]:
        print(f"Going {record[0]} to:")
        printBoardNicely(record[1])
        
def runNSample(n: int, sampleSet: list, solver):
    for _ in range(n):
        sample = sampleSet[random.randint(0, len(sampleSet)-1)]
        leafNode = solver(sample)[0]
        trackingList = traverseToRoot(leafNode)
        printTrackingNicely(trackingList)
        

# DFS
The DFS Algorithm. Takes the board and returns `(leafNode, root, totalNodes)`
As you see the queue here is a stack

In [48]:
def DFS(board: list[list[int]]):
    totalNodes = 1
    visitedNodes = set()
    root = TreeNode(None, board, "root")
    stack = [root]
    while(len(stack) != 0):
        """
        In DFS we pop from the stack
        """
        currNode = stack.pop() 
        
        currBoard = currNode.board
        
        zeroI, zeroJ = findElement(currNode.board, 0)
        visitedNodes.add(toTuple(currNode.board))
        
        if (isAnswer(currNode.board)):
           return currNode, root, totalNodes
       
        if zeroI - 1 >= 0: # up
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI - 1, zeroJ), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                stack.append(TreeNode(currNode, boardCopy, UP))
                currNode.children.append(TreeNode(currNode, boardCopy, UP))
                totalNodes += 1
        
        if zeroI + 1 < N: # Down
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI + 1, zeroJ), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                stack.append(TreeNode(currNode, boardCopy, DOWN))
                currNode.children.append(TreeNode(currNode, boardCopy, DOWN))
                totalNodes += 1
        
        if zeroJ - 1 >= 0: # left
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI, zeroJ - 1), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                stack.append(TreeNode(currNode, boardCopy, LEFT))
                currNode.children.append(TreeNode(currNode, boardCopy, LEFT))
                totalNodes += 1
        
        if zeroJ + 1 < N: # rigth
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI, zeroJ + 1), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                stack.append(TreeNode(currNode, boardCopy, RIGHT))
                currNode.children.append(TreeNode(currNode, boardCopy, RIGHT))
                totalNodes += 1

Running a sample. **We just run one easy instance since DFS is really really bad for this problem and will dig deep down the tree**

In [49]:
if RUN_ALL:
    runNSample(1, EASY_INSTANCES, DFS)

From root:
[1, 2, 3]
[4, 0, 5]
[7, 8, 6]
Going R to:
[1, 2, 3]
[4, 5, 0]
[7, 8, 6]
Going D to:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]


# BFS
The BFS Algorithm. Takes the board and returns `(leafNode, root, totalNodes)`
Note that the queue here is a simple FIFO queue

In [50]:
def BFS(board: list[list[int]]):
    totalNodes = 1
    visitedNodes = set()
    root = TreeNode(None, board, "root")
    queue = [root]
    while(len(queue) != 0):
        """
        In BFS we just get the last element in the queue (FIFO)
        """
        currNode = queue[0]
        del queue[0]
        
        currBoard = currNode.board
        
        zeroI, zeroJ = findElement(currNode.board, 0)
        visitedNodes.add(toTuple(currNode.board))
        
        if (isAnswer(currNode.board)):
           return currNode, root, totalNodes
       
        if zeroI - 1 >= 0: # up
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI - 1, zeroJ), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                queue.append(TreeNode(currNode, boardCopy, UP))
                currNode.children.append(TreeNode(currNode, boardCopy, UP))
                totalNodes += 1
                
        if zeroI + 1 < N: # Down
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI + 1, zeroJ), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                queue.append(TreeNode(currNode, boardCopy, DOWN))
                currNode.children.append(TreeNode(currNode, boardCopy, DOWN))
                totalNodes += 1
        
        if zeroJ - 1 >= 0: # left
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI, zeroJ - 1), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                queue.append(TreeNode(currNode, boardCopy, LEFT))
                currNode.children.append(TreeNode(currNode, boardCopy, LEFT))
                totalNodes += 1
      
        
        if zeroJ + 1 < N: # rigth
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI, zeroJ + 1), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                queue.append(TreeNode(currNode, boardCopy, RIGHT))
                currNode.children.append(TreeNode(currNode, boardCopy, RIGHT))
                totalNodes += 1
        

Running 2 medum samples

In [51]:
if RUN_ALL:
    runNSample(2, MEDIUM_INSTANCES, BFS)

From root:
[1, 3, 2]
[5, 4, 6]
[7, 8, 0]
Going U to:
[1, 3, 2]
[5, 4, 0]
[7, 8, 6]
Going U to:
[1, 3, 0]
[5, 4, 2]
[7, 8, 6]
Going L to:
[1, 0, 3]
[5, 4, 2]
[7, 8, 6]
Going L to:
[0, 1, 3]
[5, 4, 2]
[7, 8, 6]
Going D to:
[5, 1, 3]
[0, 4, 2]
[7, 8, 6]
Going R to:
[5, 1, 3]
[4, 0, 2]
[7, 8, 6]
Going R to:
[5, 1, 3]
[4, 2, 0]
[7, 8, 6]
Going D to:
[5, 1, 3]
[4, 2, 6]
[7, 8, 0]
Going L to:
[5, 1, 3]
[4, 2, 6]
[7, 0, 8]
Going L to:
[5, 1, 3]
[4, 2, 6]
[0, 7, 8]
Going U to:
[5, 1, 3]
[0, 2, 6]
[4, 7, 8]
Going U to:
[0, 1, 3]
[5, 2, 6]
[4, 7, 8]
Going R to:
[1, 0, 3]
[5, 2, 6]
[4, 7, 8]
Going D to:
[1, 2, 3]
[5, 0, 6]
[4, 7, 8]
Going L to:
[1, 2, 3]
[0, 5, 6]
[4, 7, 8]
Going D to:
[1, 2, 3]
[4, 5, 6]
[0, 7, 8]
Going R to:
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]
Going R to:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
From root:
[4, 3, 1]
[0, 7, 2]
[8, 5, 6]
Going R to:
[4, 3, 1]
[7, 0, 2]
[8, 5, 6]
Going U to:
[4, 0, 1]
[7, 3, 2]
[8, 5, 6]
Going R to:
[4, 1, 0]
[7, 3, 2]
[8, 5, 6]
Going D to:
[4, 1, 2]
[7, 3, 0]
[8, 

# IDS
For the IDS algorithm first we create a depth-limited DFS version and then we call it until it reaches the goal state

In [52]:
from typing import Union
def depthLimitedDFS(board: list[list[int]], depthLimit):
    totalNodes = 1
    visitedNodes = set()
    root = TreeNode(None, board, "root")
    stack = [root]
    while(len(stack) != 0):
        
        currNode = stack.pop()
        
        currBoard = currNode.board
        
        zeroI, zeroJ = findElement(currNode.board, 0)
        visitedNodes.add(toTuple(currNode.board))
        
        if (isAnswer(currNode.board)):
           return currNode, root, totalNodes
       
        if(currNode.depth < depthLimit):
            if zeroI - 1 >= 0: # up
                boardCopy = deepcopy(currBoard)
                swapItems(boardCopy, (zeroI - 1, zeroJ), (zeroI, zeroJ))
                
                boardInTuple = toTuple(boardCopy)
                if(boardInTuple not in visitedNodes):
                    stack.append(TreeNode(currNode, boardCopy, UP))
                    currNode.children.append(TreeNode(currNode, boardCopy, UP))
                    totalNodes += 1
            
            if zeroI + 1 < N: # Down
                boardCopy = deepcopy(currBoard)
                swapItems(boardCopy, (zeroI + 1, zeroJ), (zeroI, zeroJ))
                
                boardInTuple = toTuple(boardCopy)
                if(boardInTuple not in visitedNodes):
                    stack.append(TreeNode(currNode, boardCopy, DOWN))
                    currNode.children.append(TreeNode(currNode, boardCopy, DOWN))
                    totalNodes += 1
        
            
            if zeroJ - 1 >= 0: # left
                boardCopy = deepcopy(currBoard)
                swapItems(boardCopy, (zeroI, zeroJ - 1), (zeroI, zeroJ))
                
                boardInTuple = toTuple(boardCopy)
                if(boardInTuple not in visitedNodes):
                    stack.append(TreeNode(currNode, boardCopy, LEFT))
                    currNode.children.append(TreeNode(currNode, boardCopy, LEFT))
                    totalNodes += 1
        
            
            if zeroJ + 1 < N: # rigth
                boardCopy = deepcopy(currBoard)
                swapItems(boardCopy, (zeroI, zeroJ + 1), (zeroI, zeroJ))
                
                boardInTuple = toTuple(boardCopy)
                if(boardInTuple not in visitedNodes):
                    stack.append(TreeNode(currNode, boardCopy, RIGHT))
                    currNode.children.append(TreeNode(currNode, boardCopy, RIGHT))
                    totalNodes += 1
                    
                    
                    
    return False, totalNodes
            

Here we use that depth-limited-DFS

In [53]:
def IDS(board: list[list[int]]):
    depth = 1
    totalNodes = 0
    while True:
        result = depthLimitedDFS(board, depth)
        totalNodes += result[-1]
        if not isinstance(result[0], bool): # It is not False 
            return result[0], result[1], totalNodes
        depth += 1
    

Running 2 samples

In [54]:
if RUN_ALL:
    runNSample(2, MEDIUM_INSTANCES, IDS)

From root:
[2, 4, 3]
[1, 6, 5]
[8, 0, 7]
Going R to:
[2, 4, 3]
[1, 6, 5]
[8, 7, 0]
Going U to:
[2, 4, 3]
[1, 6, 0]
[8, 7, 5]
Going L to:
[2, 4, 3]
[1, 0, 6]
[8, 7, 5]
Going L to:
[2, 4, 3]
[0, 1, 6]
[8, 7, 5]
Going D to:
[2, 4, 3]
[8, 1, 6]
[0, 7, 5]
Going R to:
[2, 4, 3]
[8, 1, 6]
[7, 0, 5]
Going R to:
[2, 4, 3]
[8, 1, 6]
[7, 5, 0]
Going U to:
[2, 4, 3]
[8, 1, 0]
[7, 5, 6]
Going L to:
[2, 4, 3]
[8, 0, 1]
[7, 5, 6]
Going L to:
[2, 4, 3]
[0, 8, 1]
[7, 5, 6]
Going D to:
[2, 4, 3]
[7, 8, 1]
[0, 5, 6]
Going R to:
[2, 4, 3]
[7, 8, 1]
[5, 0, 6]
Going U to:
[2, 4, 3]
[7, 0, 1]
[5, 8, 6]
Going R to:
[2, 4, 3]
[7, 1, 0]
[5, 8, 6]
Going D to:
[2, 4, 3]
[7, 1, 6]
[5, 8, 0]
Going L to:
[2, 4, 3]
[7, 1, 6]
[5, 0, 8]
Going L to:
[2, 4, 3]
[7, 1, 6]
[0, 5, 8]
Going U to:
[2, 4, 3]
[0, 1, 6]
[7, 5, 8]
Going R to:
[2, 4, 3]
[1, 0, 6]
[7, 5, 8]
Going U to:
[2, 0, 3]
[1, 4, 6]
[7, 5, 8]
Going L to:
[0, 2, 3]
[1, 4, 6]
[7, 5, 8]
Going D to:
[1, 2, 3]
[0, 4, 6]
[7, 5, 8]
Going R to:
[1, 2, 3]
[4, 0, 6]
[7,

# UCS
The UCS Algorithm. Takes the board and returns `(leafNode, root, totalNodes)`
Note that here we use a `PriorityQueue` and set the herstric value(`node.h`) to zero for each node. And as we've mentioned we've overloaded the comparative operators for TreeNode to be compared based on `f(node) = g(node) + h(node)`

In [55]:
from queue import PriorityQueue
def UCS(board: list[list[int]]):
    totalNodes = 1
    visitedNodes = set()
    root = TreeNode(None, board, "root")
    queue = PriorityQueue()
    queue.put_nowait(root)
    while(not queue.empty()):
        
        currNode = queue.get_nowait()
        currNode: TreeNode
        
        currBoard = currNode.board
        
        zeroI, zeroJ = findElement(currNode.board, 0)
        visitedNodes.add(toTuple(currNode.board))
        
        if (isAnswer(currNode.board)):
           return currNode, root, totalNodes
       
        if zeroI - 1 >= 0: # up
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI - 1, zeroJ), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                childNode = TreeNode(currNode, boardCopy, UP)
                childNode.h = 0
                queue.put_nowait(childNode)
                currNode.children.append(childNode)
                totalNodes += 1
        
        if zeroI + 1 < N: # Down
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI + 1, zeroJ), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                childNode = TreeNode(currNode, boardCopy, DOWN)
                childNode.h = 0
                queue.put_nowait(childNode)
                currNode.children.append(childNode)
                totalNodes += 1
       
        
        if zeroJ - 1 >= 0: # left
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI, zeroJ - 1), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                childNode = TreeNode(currNode, boardCopy, LEFT)
                childNode.h = 0
                queue.put_nowait(childNode)
                currNode.children.append(childNode)
                totalNodes += 1
      
        
        if zeroJ + 1 < N: # rigth
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI, zeroJ + 1), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                childNode = TreeNode(currNode, boardCopy, RIGHT)
                childNode.h = 0
                queue.put_nowait(childNode)
                currNode.children.append(childNode)
                totalNodes += 1
        

In [56]:
if RUN_ALL:
    runNSample(2, MEDIUM_INSTANCES, UCS)

From root:
[1, 0, 2]
[7, 5, 4]
[8, 6, 3]
Going D to:
[1, 5, 2]
[7, 0, 4]
[8, 6, 3]
Going R to:
[1, 5, 2]
[7, 4, 0]
[8, 6, 3]
Going D to:
[1, 5, 2]
[7, 4, 3]
[8, 6, 0]
Going L to:
[1, 5, 2]
[7, 4, 3]
[8, 0, 6]
Going L to:
[1, 5, 2]
[7, 4, 3]
[0, 8, 6]
Going U to:
[1, 5, 2]
[0, 4, 3]
[7, 8, 6]
Going R to:
[1, 5, 2]
[4, 0, 3]
[7, 8, 6]
Going U to:
[1, 0, 2]
[4, 5, 3]
[7, 8, 6]
Going R to:
[1, 2, 0]
[4, 5, 3]
[7, 8, 6]
Going D to:
[1, 2, 3]
[4, 5, 0]
[7, 8, 6]
Going D to:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
From root:
[3, 1, 2]
[4, 5, 6]
[7, 8, 0]
Going L to:
[3, 1, 2]
[4, 5, 6]
[7, 0, 8]
Going L to:
[3, 1, 2]
[4, 5, 6]
[0, 7, 8]
Going U to:
[3, 1, 2]
[0, 5, 6]
[4, 7, 8]
Going U to:
[0, 1, 2]
[3, 5, 6]
[4, 7, 8]
Going R to:
[1, 0, 2]
[3, 5, 6]
[4, 7, 8]
Going D to:
[1, 5, 2]
[3, 0, 6]
[4, 7, 8]
Going L to:
[1, 5, 2]
[0, 3, 6]
[4, 7, 8]
Going D to:
[1, 5, 2]
[4, 3, 6]
[0, 7, 8]
Going R to:
[1, 5, 2]
[4, 3, 6]
[7, 0, 8]
Going R to:
[1, 5, 2]
[4, 3, 6]
[7, 8, 0]
Going U to:
[1, 5, 2]
[4, 3, 0]
[7, 

# A*
The last and the BEST! The A* Algorithm. Takes the board and returns `(leafNode, root, totalNodes)`
Note that also here we use a `PriorityQueue` and use both `h(node)`(heuristic) and `g(node)`(depth) for each node. And as we've mentioned we've overloaded the comparative operators for TreeNode to be compared based on `f(node) = g(node) + h(node)`

In [57]:
from queue import PriorityQueue
def AStar(board: list[list[int]]):
    totalNodes = 1
    visitedNodes = set()
    root = TreeNode(None, board, "root")
    queue = PriorityQueue()
    queue.put_nowait(root)
    while(not queue.empty()):
        
        currNode = queue.get_nowait()
        currNode: TreeNode
        currBoard = currNode.board
        
        zeroI, zeroJ = findElement(currNode.board, 0)
        visitedNodes.add(toTuple(currNode.board))
        
        if (isAnswer(currNode.board)):
           return currNode, root, totalNodes
       
        if zeroI - 1 >= 0: # up
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI - 1, zeroJ), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                childNode = TreeNode(currNode, boardCopy, UP)
                queue.put_nowait(childNode)
                currNode.children.append(childNode)
                totalNodes += 1
        
        if zeroI + 1 < N: # Down
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI + 1, zeroJ), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                childNode = TreeNode(currNode, boardCopy, DOWN)
                queue.put_nowait(childNode)
                currNode.children.append(childNode)
                totalNodes += 1
       
        
        if zeroJ - 1 >= 0: # left
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI, zeroJ - 1), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                childNode = TreeNode(currNode, boardCopy, LEFT)
                queue.put_nowait(childNode)
                currNode.children.append(childNode)
                totalNodes += 1
      
        
        if zeroJ + 1 < N: # rigth
            boardCopy = deepcopy(currBoard)
            swapItems(boardCopy, (zeroI, zeroJ + 1), (zeroI, zeroJ))
            
            boardInTuple = toTuple(boardCopy)
            if(boardInTuple not in visitedNodes):
                childNode = TreeNode(currNode, boardCopy, RIGHT)
                queue.put_nowait(childNode)
                currNode.children.append(childNode)
                totalNodes += 1
        

In [58]:
if RUN_ALL:
    runNSample(2, HARD_INSTANCES, AStar)

From root:
[1, 3, 5]
[8, 7, 0]
[6, 2, 4]
Going D to:
[1, 3, 5]
[8, 7, 4]
[6, 2, 0]
Going L to:
[1, 3, 5]
[8, 7, 4]
[6, 0, 2]
Going L to:
[1, 3, 5]
[8, 7, 4]
[0, 6, 2]
Going U to:
[1, 3, 5]
[0, 7, 4]
[8, 6, 2]
Going R to:
[1, 3, 5]
[7, 0, 4]
[8, 6, 2]
Going R to:
[1, 3, 5]
[7, 4, 0]
[8, 6, 2]
Going D to:
[1, 3, 5]
[7, 4, 2]
[8, 6, 0]
Going L to:
[1, 3, 5]
[7, 4, 2]
[8, 0, 6]
Going L to:
[1, 3, 5]
[7, 4, 2]
[0, 8, 6]
Going U to:
[1, 3, 5]
[0, 4, 2]
[7, 8, 6]
Going R to:
[1, 3, 5]
[4, 0, 2]
[7, 8, 6]
Going R to:
[1, 3, 5]
[4, 2, 0]
[7, 8, 6]
Going U to:
[1, 3, 0]
[4, 2, 5]
[7, 8, 6]
Going L to:
[1, 0, 3]
[4, 2, 5]
[7, 8, 6]
Going D to:
[1, 2, 3]
[4, 0, 5]
[7, 8, 6]
Going R to:
[1, 2, 3]
[4, 5, 0]
[7, 8, 6]
Going D to:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
From root:
[4, 5, 3]
[2, 1, 0]
[8, 7, 6]
Going U to:
[4, 5, 0]
[2, 1, 3]
[8, 7, 6]
Going L to:
[4, 0, 5]
[2, 1, 3]
[8, 7, 6]
Going D to:
[4, 1, 5]
[2, 0, 3]
[8, 7, 6]
Going L to:
[4, 1, 5]
[0, 2, 3]
[8, 7, 6]
Going D to:
[4, 1, 5]
[8, 2, 3]
[0, 

### Now we just define some useful functions to make things easier to run algorithms on all the instances and store the results data 

In [18]:
def solveInstance(board: list[list[int]], solver):
    start = time.perf_counter()
    result = solver(board)
    return result, time.perf_counter() - start

In [19]:
def reportInstanceSet(instanceSet: list, solver):
    totalTime = 0
    totalNodes = 0
    totalDepths = 0
    times = []
    nodes = []
    depths = []
    for instance in instanceSet:
        result, timeSpent = solveInstance(instance, solver)
        
        nodes.append(result[-1])
        totalNodes += result[-1]
        
        times.append(timeSpent)
        totalTime += timeSpent
        
        depths.append(len(traverseToRoot(result[0])))
        totalDepths += depths[-1]
        
        
    return nodes, totalNodes, times, totalTime, depths, totalDepths


## RUN ALL
#### Here we run all the algorithm on all instances and store the results(time, space, depth)

In [20]:
import pandas as pd
algs = {"DFS": DFS, "BFS": BFS, "IDS": IDS, "UCS": UCS, "AStar": AStar}
levels = ["EASY", "MEDIUM", "HARD"]
algInfos = dict()

In [21]:
import numpy as np
for alg in algs:
    info = {}
    for level, instanceSet in zip(levels, [EASY_INSTANCES, MEDIUM_INSTANCES, HARD_INSTANCES]):
        print(f"Running {alg} on {level}")
        info[level] = dict()
        info[level]["details"] = []
        info[level]["summary"] = []
        nodes, totalNodes, times, totalTimes, depths, totalDepths = reportInstanceSet(instanceSet, algs[alg])
        for instanceNum in range(len(instanceSet)):
            info[level]["details"].append([f"{alg} in {level}({instanceNum})", times[instanceNum], nodes[instanceNum], depths[instanceNum]-1])

        info[level]["summary"] = [np.average(times), np.average(nodes), np.average(depths)]
    algInfos[alg] = info
    

Running DFS on EASY
Running DFS on MEDIUM
Running DFS on HARD
Running BFS on EASY
Running BFS on MEDIUM
Running BFS on HARD
Running IDS on EASY
Running IDS on MEDIUM
Running IDS on HARD
Running UCS on EASY
Running UCS on MEDIUM
Running UCS on HARD
Running AStar on EASY
Running AStar on MEDIUM
Running AStar on HARD


## Result Summary
Here we draw a table of summary of results

In [22]:
from tabulate import tabulate

summary = [["Alg | Level", "Time", "Space", "Depth(steps)"]]
for alg in algs:
    for level in levels:
        summary.append([f"{alg} in {level}", *algInfos[alg][level]["summary"]])

print(tabulate(summary, tablefmt="double_grid"))

╔═════════════════╦═══════════════════════╦════════════════════╦════════════════════╗
║ Alg | Level     ║ Time                  ║ Space              ║ Depth(steps)       ║
╠═════════════════╬═══════════════════════╬════════════════════╬════════════════════╣
║ DFS in EASY     ║ 1.7530593227009377    ║ 56189.95           ║ 32193.75           ║
╠═════════════════╬═══════════════════════╬════════════════════╬════════════════════╣
║ DFS in MEDIUM   ║ 4.027051039133221     ║ 112610.6           ║ 64440.6            ║
╠═════════════════╬═══════════════════════╬════════════════════╬════════════════════╣
║ DFS in HARD     ║ 4.643542599301145     ║ 133487.85          ║ 75584.0            ║
╠═════════════════╬═══════════════════════╬════════════════════╬════════════════════╣
║ BFS in EASY     ║ 0.008179454749188153  ║ 352.7              ║ 7.15               ║
╠═════════════════╬═══════════════════════╬════════════════════╬════════════════════╣
║ BFS in MEDIUM   ║ 0.8192213386670725    ║ 21766.4666

## Conclusions
- As you can see A* is the best in any case and in any factor. 

- Between BFS and DFS, when the solution is in depth(hard instances) DFS may be a better solution cause BFS needs to build the complete tree until that depth so it's much worse in space and time

- Another reason that DFS is bad is that it seaches the depth for solution and deeper means longer solution. For example a solution in depth of 6 requires 6 moves to get to the goal state but a solution to the same instance in depth of 1246 require 1246 moves to get to the goal state.

- Also time with IDS is similar to BFS in EASY and MEDIUM instances.

- The space with IDS is also so bad cause it has to build the tree each time it deepens

- The UCS is also similar to BFS in time and space.

- A* is the best of all. since it uses more information(both heustric and depth) and uses a PriorityQueue. 

- And we've also proved that A* give the optimal answer


## And finally create a `pd.DataFrame` out of the details to store in an `.xlsx` file

In [23]:
maindf = pd.DataFrame()
for alg in algs:
    for level in levels:
        maindf = pd.concat((maindf, pd.DataFrame(algInfos[alg][level]["details"])))
        
maindf.columns = ["Alg | Level", "Time", "Space", "Depth(steps)"]
maindf

,Alg | Level,Time,Space,Depth(steps)
0,DFS in EASY(0),0.003130,104,55
1,DFS in EASY(1),2.255104,89876,51428
2,DFS in EASY(2),2.206512,83408,47661
3,DFS in EASY(3),4.686545,173143,99665
4,DFS in EASY(4),5.084924,190649,108818
...,...,...,...,...
15,AStar in HARD(15),0.315839,12626,27
16,AStar in HARD(16),0.321154,12792,28
17,AStar in HARD(17),0.142727,5616,27
18,AStar in HARD(18),0.182133,7336,25


In [24]:
maindf.to_excel("Details.xlsx")